In [ ]:
%matplotlib inline

import torch
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from data_loaders import train_loader, test_loader
from utils_visualization import visualize_data_loader
from custom_resnet import Net
from train_test import train, test
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)



classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

means=[0.485,0.456,0.406]
stds=[0.229,0.224,0.225]
batch_size = 512
num_workers = 32
cuda_available = torch.cuda.is_available()
cuda_available

In [ ]:
trainldr = train_loader(batch_size, num_workers, cuda_available, means, stds)
testldr = test_loader(batch_size, num_workers, cuda_available, means, stds)

In [ ]:
dataiter = iter(trainldr)
images, labels = next(dataiter)


visualize_data_loader(images, labels, classes, 5)

In [ ]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
net = Net('bn').to(device)
summary(net, input_size=(3, 32, 32))

In [ ]:
import torch.nn as nn
from torch_lr_finder import LRFinder
train_criterion = nn.CrossEntropyLoss()
test_criterion = nn.CrossEntropyLoss(reduction='sum')
# train_criterion = F.nll_loss
# test_criterion = F.nll_loss
optimizer = torch.optim.Adam(net.parameters(), lr=0.03, weight_decay=1e-4)
lr_finder = LRFinder(net, optimizer, train_criterion, device='cuda')
lr_finder.range_test(trainldr,end_lr=10, num_iter=200, step_mode='exp')
lr_finder.plot()
lr_finder.reset()

In [ ]:
import torch
from tqdm import tqdm
from torch.optim import lr_scheduler

EPOCHS = 24

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

scheduler = lr_scheduler.OneCycleLR(optimizer,
                                    max_lr=4.93E-02,
                                    steps_per_epoch=len(trainldr),
                                    epochs=EPOCHS,
                                    pct_start=5/EPOCHS,
                                    div_factor=100,
                                    final_div_factor=100,
                                    three_phase=False,
                                    anneal_strategy='linear')


train_losses = []
test_losses = []
train_acc = []
test_acc = []

# model =  Net('bn').to(device)

# learning_rate = []
EPOCHS = 24
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    for param_group in optimizer.param_groups:
      print("lr= ",param_group['lr'])
    train_losses, train_acc = train(net, device, trainldr, optimizer, epoch, scheduler, train_criterion, train_losses, train_acc)
    test_loss, test_acc = test(net, device, testldr,test_criterion, test_losses, test_acc)
print('Finished Training')

